In [1]:
import boto3
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def read_json_files_from_s3(bucket_name, prefix=None, max_files=None):
    # Inicializa o recurso S3
    s3r = boto3.resource('s3')

    # Referência ao bucket S3
    bucket = s3r.Bucket(bucket_name)

    # Armazena os conteúdos dos arquivos JSON
    json_files_contents = []

    # Variável para contar o número de arquivos processados
    file_count = 0

    # Itera sobre os objetos no bucket, com o prefixo (se fornecido)
    if prefix:
        objects = bucket.objects.filter(Prefix=prefix)
    else:
        objects = bucket.objects.all()

    # Processa os arquivos do bucket
    for obj in objects:
        # Verifica se o arquivo é JSON
        if obj.key.endswith('.json'):
            # Obtém o conteúdo do arquivo
            file_content = obj.get()['Body'].read().decode('utf-8')

            # Converte o conteúdo do JSON para um dicionário Python e armazena
            json_data = json.loads(file_content)
            json_files_contents.append(json_data)  # Armazena como dicionário

            # Incrementa o contador de arquivos processados
            file_count += 1

            # Verifica se atingiu o limite de arquivos
            if max_files and file_count >= max_files:
                break

    # Converte a lista de dicionários em um DataFrame
    df = pd.DataFrame(json_files_contents)

    return df

In [ ]:
json_data = read_json_files_from_s3('match-info',max_files=10263)

/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [ ]:
processed_data = []

# Itera por cada linha (partida)
for index, row in json_data.iterrows():
    if 'CLASSIC' in row['info']['gameMode']:
#         print(row['info']['gameMode'])
        match_data = {}

        participants = row['info']['participants']
#         print(participants)
        teams = row['info']['teams']  
        for i, participant in enumerate(participants):
#             kills = participant['kills']
#             deaths = participant['deaths']
#             assists = participant['assists']        

            
            if 'challenges' in participant:
                kda = participant['challenges']['kda']
                gold_per_minute = participant['challenges']['goldPerMinute']
            
            vision_score = participant['visionScore']

            match_data[f'player_{i+1}_kda'] = kda
            match_data[f'player_{i+1}_gold_per_minute'] = gold_per_minute
            match_data[f'player_{i+1}vision_score'] = vision_score

        team_100 = next(team for team in teams if team['teamId'] == 100)
        match_data['team_100_won'] = 1 if team_100['win'] else 0

        processed_data.append(match_data)

processed_df = pd.DataFrame(processed_data)

csv_buffer = StringIO()
processed_df.to_csv(csv_buffer, index=False)

s3 = boto3.client('s3')

# Nome do bucket e chave (nome do arquivo no S3)
bucket_name = 'match-dataset'
s3_key = 'matchs.csv'

# Salva o CSV no S3
s3.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())

print("CSV criado e salvo no S3 com sucesso!")

In [1]:
from io import StringIO
import boto3
import pandas as pd
s3_client = boto3.client('s3')

/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [5]:
 # Baixar o arquivo do S3
csv_file = s3_client.get_object(Bucket='match-dataset', Key='matchs.csv')
    
# Ler o conteúdo do arquivo como uma string
csv_content = csv_file['Body'].read().decode('utf-8')
    
 # Converter para um DataFrame usando StringIO
df = pd.read_csv(StringIO(csv_content))

In [6]:
X = df.drop(columns=['team_100_won'])
y = df['team_100_won']

In [8]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# Criar o modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [19]:
# Treinar o modelo
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [11]:
# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar a performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy}')

Acurácia: 0.9349794238683128


In [17]:
nova_entrada = [
[3.3988690476190477, 471.9657343736383, 0], 
[9, 414.0098336382496, 46.44444444444444], 
[5.111666666666667, 435.4742655669012, 0],
[1, 363.4293092847436, 29.85], 
[1.3333333333333333, 416.8990610164101, 17.95], 
[2.8768831168831173, 533.3127612365831, 0], 
[3.0214814814814814, 322.24319249188727, 67.53333333333333],
]

In [18]:
# Transformar a entrada em um array unidimensional (como está no seu exemplo original)
nova_entrada_unidimensional = [item for sublist in nova_entrada for item in sublist]

# Fazer predição com o modelo treinado
predicao = model.predict([nova_entrada_unidimensional])
print(f'Predição para a nova entrada: {predicao}')

/usr/local/lib64/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


ValueError: X has 21 features, but RandomForestClassifier is expecting 30 features as input.

In [17]:
import boto3

# Inicializa o cliente S3
s3 = boto3.client('s3')

def contar_itens_bucket(bucket_name):
    # Lista todos os objetos no bucket
    response = s3.list_objects_v2(Bucket=bucket_name)
    
    # Verifica se o bucket contém objetos
    if 'Contents' not in response:
        return 0
    
    # Conta os objetos
    total_itens = len(response['Contents'])
    
    # Verifica se há mais páginas de objetos (paginado)
    while response.get('IsTruncated'):  # Se o resultado estiver truncado, há mais objetos a serem recuperados
        response = s3.list_objects_v2(Bucket=bucket_name, ContinuationToken=response['NextContinuationToken'])
        total_itens += len(response['Contents'])
    
    return total_itens

# Nome do seu bucket S3
bucket_name = 'match-info'

# Exibe o total de itens
total = contar_itens_bucket(bucket_name)
print(f'Total de itens no bucket {bucket_name}: {total}')


/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Total de itens no bucket match-info: 10263


In [21]:
import joblib
joblib.dump(model, 'modelo_random_forest2.pkl')

['modelo_random_forest2.pkl']

In [28]:
import pickle
with open('/home/ec2-user/modelo_random_forest3.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [26]:
!pip3 install --upgrade scikit-learn==1.5.2



ERROR: Ignored the following versions that require a different python version: 1.1.0 Requires-Python >=3.8; 1.1.1 Requires-Python >=3.8; 1.1.2 Requires-Python >=3.8; 1.1.3 Requires-Python >=3.8; 1.2.0 Requires-Python >=3.8; 1.2.0rc1 Requires-Python >=3.8; 1.2.1 Requires-Python >=3.8; 1.2.2 Requires-Python >=3.8; 1.3.0 Requires-Python >=3.8; 1.3.0rc1 Requires-Python >=3.8; 1.3.1 Requires-Python >=3.8; 1.3.2 Requires-Python >=3.8; 1.4.0 Requires-Python >=3.9; 1.4.0rc1 Requires-Python >=3.9; 1.4.1.post1 Requires-Python >=3.9; 1.4.2 Requires-Python >=3.9; 1.5.0 Requires-Python >=3.9; 1.5.0rc1 Requires-Python >=3.9; 1.5.1 Requires-Python >=3.9; 1.5.2 Requires-Python >=3.9
ERROR: Could not find a version that satisfies the requirement scikit-learn==1.5.2 (from versions: 0.9, 0.10, 0.11, 0.12, 0.12.1, 0.13, 0.13.1, 0.14, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.17, 0.17.1, 0.18, 0.18.1, 0.18.2, 0.19.0, 0.19.1, 0.19.2, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.20.4, 0.21.0, 0.21.1, 0.21.2, 0.